## Backprojetion from triangulation

In [1]:
# import necessary modules
import matplotlib.pyplot as plt
import numpy as np

import multicam_calibration as mcc
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import plotly.express as px
import pickle

In [2]:
from threed_utils.visualization import plot_triangulations, plot_back_projections
from threed_utils.old_triangulation import back_project_points
from utils import read_first_frame

### get triangulation from predictions: mcc, anipose

In [4]:
# get data from one session:
from pathlib import Path
dir_path = Path(r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\20240724\100455\multicam_video_2024-07-24T10_20_07_cropped_20241209165236")
calibration_path = Path(r"D:\P05_3DRIG_YE-LP\e01_mouse_hunting\v04_mice-hunting\20240724\calibration\multicam_video_2024-07-24T14_13_45_cropped_20241209165236")

### Get calibration matrices

In [3]:
data_dir = Path('/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/calibration')

In [4]:
# get the matrices:

from threed_utils.io import movement_ds_from_anipose_triangulation_df, read_calibration_toml


In [5]:
calibration_paths = sorted(data_dir.glob("mc_calibration_output_*"))
last_calibration_path = calibration_paths[-1]

all_calib_uvs = np.load(last_calibration_path / "all_calib_uvs.npy")
calib_toml_path = last_calibration_path / "calibration_from_mc.toml"
print(calib_toml_path)
cam_names, img_sizes, extrinsics, intrinsics = read_calibration_toml(calib_toml_path)

/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/calibration/mc_calibration_output_20241210-164414/calibration_from_mc.toml


In [6]:
# get triangulated points from both: 
import xarray as xr

slp_dir = Path('/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/data/video_test')

triangulation_path = slp_dir / 'anipose_triangulated_ds.h5'
mcc_path = slp_dir / 'mcc_triangulated_ds.h5'
anipose_ds = xr.open_dataset(triangulation_path)
mcc_ds = xr.open_dataset(mcc_path)


In [8]:
arena_path = '/Users/thomasbush/Documents/Vault/Iurilli_lab/3d_tracking/3d-setup/tests/assets/arena_tracked_points.pkl'
with open(arena_path, 'rb') as f:
    arena_points = pickle.load(f)

In [ ]:
# get calibration data from 2_calibration_multicam_script.py on calibration_path (just change path inside) [DONE]

# store calibration data for backprojection[DONE]


# use calibration data on 3_triangulation_mcc_anipose_script.py to obtain triangulation data for both [DONE]


# modify backprojection to work with xarrays 

# once backprojection works, create function with plotting


# unify the two into a single script


# Resources: videos, calibration matrices, triangulation data



## Backprojection of triangulated points:

def backproject_triangulated_points(xarray_ds, extrinsics, intrinsics, frame_n):


